In [41]:
import requests as req
import pandas as pd
import json
# Please make sure your python version is 3.6 or lower
# https://github.com/tweepy/tweepy/issues/1017
import tweepy

### Gather data

In this part, I am going to be downloading and saving to disk `image-predictions.tsv` data from Udacity provided URL

In [25]:
# Download file
image_predictions = req.get(
    "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
).text

In [29]:
# write to disk
with open("image-predictions.tsv", "w") as f:
    f.write(image_predictions)

Loading data from twitter and storing them in `tweet_json.txt`

**Please set your credentials in `credentials.json` file to download data from twitter api**

In [42]:
# Load credentials an authorize
with open('credentials.json') as f:
    creds = json.load(f)

consumer_key = creds["consumer_key"]
consumer_secret = creds["consumer_secret"]
access_token = creds["access_token"]
access_secret = creds["access_secret"]
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

### Load datasets

In [23]:
img_predictions = pd.read_csv("image-predictions.tsv", sep="\t")

In [24]:
img_predictions.head()


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
